In [1]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def doublefactorial(n):
    if n==0:return 1
    if n>0:
        def doublefactorialEven(n):
            return 2^(n/2)*ZZ(factorial(n/2))
        def doublefactorialOdd(n):
            k=(n+1)/2
            return (factorial(2*k))/(2^k*factorial(k))
        if 2 in divisors(n):return doublefactorialEven(n)
        if 2 in divisors(n+1):return doublefactorialOdd(n)
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)
def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)
def j(n,m):return L(xjNew(n,m)/x)
def polynomialCoefficient(n,poly):
    return poly.list()[n]
def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    return polynomialCoefficient(dg,polynomial)
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})
def lcmlist(list):
    answer=list[0]
    for k in [1..len(list)-1]:
        answer=lcm(answer,list[k])
    return answer
def denominators(list):
    dens=[]
    lnth=len(list)
    for k in [0..lnth-1]:
        dens=dens+[denominator(list[k])]
    return dens
def normalpolynomial(poly):
    return poly*lcmlist(denominators(poly.coefficients()))

In [4]:
print(j(5,3));print("");print(J(5,3))

x^-1 + 744 + 196884*x + 21493760*x^2 + 864299970*x^3 + 20245856256*x^4

x^-1 + 31/72 + 1823/27648*x + 10495/2519424*x^2 + 1778395/18345885696*x^3 + 45767/34828517376*x^4 + 41650330075/3327916660110655488*x^5 + O(x^6)


In [14]:
poly=xjNew(5,3);
print(poly)
print(poly.degree())
print list(poly)

20245856256*x^5 + 864299970*x^4 + 21493760*x^3 + 196884*x^2 + 744*x + 1
5
[1, 744, 196884, 21493760, 864299970, 20245856256]


In [3]:
print j(5,3)

x^-1 + 744 + 196884*x + 21493760*x^2 + 864299970*x^3 + 20245856256*x^4


In [2]:
data=[]
for m in [3..500]:
    if 10 in divisors(m):
        print(m)
    data=data+[[m,j(120,m)]]
import pickle
wfile = open('/Users/barrybrent/8mar21no6.txt','a')
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500


In [4]:
import pickle
rfile = open('/Users/barrybrent/8mar21no6.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
polydata=[]
import time
wfile = open('/Users/barrybrent/10mar21no8.txt','w')
enoughfile=[]
for qpower in [0..120]:
    start = time.time()
    data=[]
    for k in [0..len(s)-1]:
        m = s[k][0]
        poly=x*s[k][1] 
# times x because, to use polynomialCoefficient, I need a polynomial; not a Laurent series
        cf=polynomialCoefficient(qpower,poly)
        data=data+[[m,cf]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[qpower-1,rl]] 
    finish = time.time()
    print([qpower,finish-start])
# -1 because I want to record the coefficients of the qpowers of the original Laurent series
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()

[0, 3.476809024810791]
[1, 3.5383589267730713]
[2, 3.4991610050201416]
[3, 3.594343900680542]
[4, 3.532088041305542]
[5, 3.5358738899230957]
[6, 3.5763092041015625]
[7, 3.5106911659240723]
[8, 3.568726062774658]
[9, 3.5482139587402344]
[10, 3.553571939468384]
[11, 3.594465970993042]
[12, 3.520862102508545]
[13, 3.532066822052002]
[14, 3.661604166030884]
[15, 3.5511369705200195]
[16, 3.592189073562622]
[17, 3.567767858505249]
[18, 3.59506893157959]
[19, 3.631237030029297]
[20, 3.5660011768341064]
[21, 3.608276844024658]
[22, 3.559959888458252]
[23, 3.584864854812622]
[24, 3.6208600997924805]
[25, 3.5829501152038574]
[26, 3.615128993988037]
[27, 3.6606009006500244]
[28, 3.621027946472168]
[29, 3.6795859336853027]
[30, 3.7123470306396484]
[31, 3.688579797744751]
[32, 3.6764211654663086]
[33, 3.658989191055298]
[34, 3.6817078590393066]
[35, 3.6548259258270264]
[36, 3.682342767715454]
[37, 3.721776008605957]
[38, 3.702333927154541]
[39, 3.7165682315826416]
[40, 3.744623899459839]
[41, 3.689

In [104]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
for n in [1..5]:
    print(s[n][0])
    f=D(s[n][1])
    lf=lastfactor(f)
    np=normalpolynomial(lf)
    print(lf);print(np)
    print("---------------------------------------------------------------------------------")

0
x^2 + 4/3
3*x^2 + 4
---------------------------------------------------------------------------------
1
x^4 - 8/69*x^2 - 16/23
69*x^4 - 8*x^2 - 48
---------------------------------------------------------------------------------
2
x^4 - 8/27*x^2 - 16/27
27*x^4 - 8*x^2 - 16
---------------------------------------------------------------------------------
3
x^6 - 38732/5601*x^4 + 11312/5601*x^2 + 6464/1867
5601*x^6 - 38732*x^4 + 11312*x^2 + 19392
---------------------------------------------------------------------------------
4
x^8 - 51968/3375*x^6 + 650144/10125*x^4 - 190976/10125*x^2 - 95488/3375
10125*x^8 - 155904*x^6 + 650144*x^4 - 190976*x^2 - 286464
---------------------------------------------------------------------------------


In [116]:
data=[]
for n in [1..20]:
    data=data+[doublefactorial(n)]
print(data)

[1, 2, 3, 8, 15, 48, 105, 384, 945, 3840, 10395, 46080, 135135, 645120, 2027025, 10321920, 34459425, 185794560, 654729075, 3715891200]


In [2]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
k=1
n=s[k][0]
print([s[k][0],2^n*ZZ(factorial(n)),doublefactorial(2*n)])
f=D(s[k][1])
lf=lastfactor(f)
np=normalpolynomial(lf)
print(np)

[0, 1, 1]
3*x^2 + 4


In [ ]:
Magma V2.25-8     Thu Mar 18 2021 09:19:52 on Barrys-iMac [Seed = 522985201]
Type ? for help.  Type <Ctrl>-D to quit.
> P< x >:=PolynomialAlgebra(Rationals());
> f:=-3*x^2 + 4;
> G:=GaloisGroup(f);
> print G;
Symmetric group G acting on a set of cardinality 2
Order = 2


In [3]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
k=2
n=s[k][0]
print([s[k][0],2^n*ZZ(factorial(n)),doublefactorial(2*n)])
f=D(s[k][1])
lf=lastfactor(f)
np=normalpolynomial(lf)
print(np)

[1, 2, 2]
69*x^4 - 8*x^2 - 48


In [ ]:
> P< x >:=PolynomialAlgebra(Rationals());
> f:=69*x^4 - 8*x^2 - 48;
> G:=GaloisGroup(f);
> print G;
Permutation group G acting on a set of cardinality 4
Order = 8 = 2^3
    (1, 2)(3, 4)
    (2, 3)

In [4]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
k=3
n=s[k][0]
print([s[k][0],2^n*ZZ(factorial(n)),doublefactorial(2*n)])
f=D(s[k][1])
lf=lastfactor(f)
np=normalpolynomial(lf)
print(np)

[2, 8, 8]
27*x^4 - 8*x^2 - 16


In [ ]:
> P< x >:=PolynomialAlgebra(Rationals());
> f:=27*x^4 - 8*x^2 - 16
> G:=GaloisGroup(f);
> print G;
Permutation group G acting on a set of cardinality 4
Order = 8 = 2^3
    (1, 2)(3, 4)
    (2, 3)

In [5]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
k=4
n=s[k][0]
print([s[k][0],2^n*ZZ(factorial(n)),doublefactorial(2*n)])
f=D(s[k][1])
lf=lastfactor(f)
np=normalpolynomial(lf)
print(np)

[3, 48, 48]
5601*x^6 - 38732*x^4 + 11312*x^2 + 19392


In [ ]:
Magma V2.25-8     Thu Mar 18 2021 11:56:56 on Barrys-iMac [Seed = 3805349073]
Type ? for help.  Type <Ctrl>-D to quit.
> P< x >:=PolynomialAlgebra(Rationals());
> f:=5601*x^6 - 38732*x^4 + 11312*x^2 + 19392;
> G:=GaloisGroup(f);
> print G;
Permutation group G acting on a set of cardinality 6
Order = 48 = 2^4 * 3
    (1, 2, 3)(4, 5, 6)
    (1, 2)(4, 5)
    (1, 4)


In [6]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
k=5
n=s[k][0]
print([s[k][0],2^n*ZZ(factorial(n)),doublefactorial(2*n)])
f=D(s[k][1])
lf=lastfactor(f)
np=normalpolynomial(lf)
print(np)

[4, 384, 384]
10125*x^8 - 155904*x^6 + 650144*x^4 - 190976*x^2 - 286464


In [ ]:
> P< x >:=PolynomialAlgebra(Rationals());
> f:=10125*x^8 - 155904*x^6 + 650144*x^4 - 190976*x^2 - 286464;
> G:=GaloisGroup(f);
> print G;
Permutation group G acting on a set of cardinality 8
Order = 384 = 2^7 * 3
    (1, 4, 2, 3)(5, 8, 6, 7)
    (2, 3)(6, 7)
    (2, 6)


In [7]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
k=6
n=s[k][0]
print([s[k][0],2^n*ZZ(factorial(n)),doublefactorial(2*n)])
f=D(s[k][1])
lf=lastfactor(f)
np=normalpolynomial(lf)
print(np)

[5, 3840, 3840]
16769187*x^10 - 419821596*x^8 + 3737957344*x^6 - 12223806336*x^4 + 3651604224*x^2 + 4868805632


In [ ]:
> P< x >:=PolynomialAlgebra(Rationals());                                                                                                                                                               
> f:=16769187*x^10-419821596*x^8+3737957344*x^6-12223806336*x^4+3651604224*x^2+4868805632;
> G:=GaloisGroup(f);
> print G;
Permutation group G acting on a set of cardinality 10
Order = 3840 = 2^8 * 3 * 5
    (1, 2, 7, 8, 3)(4, 5, 9, 10, 6)
    (1, 3)(4, 6)
    (3, 6)
> 


In [8]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
k=7
n=s[k][0]
print([s[k][0],2^n*ZZ(factorial(n)),doublefactorial(2*n)])
f=D(s[k][1])
lf=lastfactor(f)
np=normalpolynomial(lf)
print(np)

[6, 46080, 46080]
28940625*x^12 - 1021579752*x^10 + 14381852336*x^8 - 97244606208*x^6 + 275976533760*x^4 - 84389242880*x^2 - 101267091456


In [ ]:
> f:=28940625*x^12-1021579752*x^10+14381852336*x^8-97244606208*x^6+275976533760*x^4-84389242880*x^2-101267091456;
> G:=GaloisGroup(f);
> print G;
Permutation group G acting on a set of cardinality 12
Order = 46080 = 2^10 * 3^2 * 5
    (1, 2, 5, 7, 8, 6)(3, 4, 9, 11, 12, 10)
    (7, 8)(11, 12)
    (7, 11)


In [9]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
k=8
n=s[k][0]
print([s[k][0],2^n*ZZ(factorial(n)),doublefactorial(2*n)])
f=D(s[k][1])
lf=lastfactor(f)
np=normalpolynomial(lf)
print(np)

[7, 645120, 645120]
171449044875*x^14 - 7888431575988*x^12 + 153388981660272*x^10 - 1601285210822720*x^8 + 9150173038346496*x^6 - 23764850390670336*x^4 + 7468858536415232*x^2 + 8147845676089344


In [ ]:
> f:=171449044875*x^14-7888431575988*x^12+153388981660272*x^10-1601285210822720*x^8+9150173038346496*x^6-23764850390670336*x^4+7468858536415232*x^2+8147845676089344;
> G:=GaloisGroup(f);
> print G;
Permutation group G acting on a set of cardinality 14
Order = 645120 = 2^11 * 3^2 * 5 * 7
    (1, 7, 3, 9, 10, 2, 8)(4, 11, 6, 13, 14, 5, 12)
    (1, 7)(4, 11)
    (1, 4)


In [10]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
k=9
n=s[k][0]
print([s[k][0],2^n*ZZ(factorial(n)),doublefactorial(2*n)])
f=D(s[k][1])
lf=lastfactor(f)
np=normalpolynomial(lf)
print(np)

[8, 10321920, 10321920]
11533417875*x^16 - 659279330928*x^14 + 16469761126016*x^12 - 233226372227840*x^10 + 2013551386772992*x^8 - 10287291525124096*x^6 + 25262578868092928*x^4 - 8182074782580736*x^2 - 8182074782580736


In [ ]:
> f:=11533417875*x^16-659279330928*x^14+16469761126016*x^12-233226372227840*x^10+2013551386772992*x^8-10287291525124096*x^6+25262578868092928*x^4-8182074782580736*x^2-8182074782580736;
> G:=GaloisGroup(f);
> print G;
Permutation group G acting on a set of cardinality 16
Order = 10321920 = 2^15 * 3^2 * 5 * 7
    (1, 6, 5, 10, 8, 7, 9, 2)(3, 12, 11, 16, 14, 13, 15, 4)
    (5, 10)(11, 16)
    (5, 11)
